In [1]:
import psycopg2
import datetime
import json
from dateutil.parser import parse
from dateutil.tz import gettz
import time

In [2]:
conn = psycopg2.connect("dbname = template_postgis user = postgres\
                       host = localhost password = 141421")
cur = conn.cursor()
cur.execute("""
SELECT srtext FROM spatial_ref_sys
WHERE srid = 32610;
""")
rows = cur.fetchall()
print(rows[0])

('PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]',)


In [1]:
!pip install psycopg2

## Load Tweeter data

In [3]:
with open('tweets/tweets_1M.json') as data_file:    
    data = json.load(data_file)
data = data[:100000]

In [4]:
for i in data:
    print(i)
    break

{'user_id': 224874450, 'text': '@Tanner_Cortez hey checkout the website: http://t.co/LuktJ6hiws', 'timeStamp': 'Wed Sep 11 04:38:08 +0000 2013', 'lat': 37.44609999, 'lng': -121.88355687, 'id': 377652254096228352}


## Establish connection

In [5]:
conn = psycopg2.connect("dbname= tweets_db user= postgres host=localhost password=141421")
cur = conn.cursor()
conn.commit()

In [6]:
# Verify your installation
conn.rollback()
cur = conn.cursor()
cur.execute("SELECT srtext FROM spatial_ref_sys WHERE srid = 32610;")
rows = cur.fetchall()
print(rows[0])

('PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]',)


## 4.2 Create a table

In [24]:
try:
    cur.execute("DROP TABLE tweets")
except:
    conn.rollback()
cur.execute("""
CREATE TABLE tweets
(
id VARCHAR,


user_id VARCHAR,
loc geometry(POINT),
time VARCHAR,
text VARCHAR
);
""")

In [37]:
from dateutil.parser import parse

## 4.3 Insert tweeter records

In [25]:
for i in data:
    time = parse(i['timeStamp'])
    cur.execute("INSERT INTO tweets(id, user_id, loc, time, text) VALUES(%s, %s, ST_SetSRID(ST_MakePOINT(%s,%s),%s), %s, %s)",
                (i['id'], i['user_id'], i['lng'], i['lat'],'4326', time, i['text']))

In [13]:
cur.execute('SELECT * from tweets')
rows = cur.fetchall()
print(rows[0])

('377652254096228352', '224874450', '0101000020E6100000E1E5F1CD19B9424098331D328C785EC0', '2013-09-10 21:38:08-07', '@Tanner_Cortez hey checkout the website: http://t.co/LuktJ6hiws')


## 4.5 Create spatial index

In [14]:
"""
TODO
"""
cur.execute("CREATE INDEX spatial_idx ON ca_census_tract USING GIST(geom);")

## 4.6 Create id index for tweets table

In [15]:
"""
TODO
"""
cur.execute("CREATE INDEX id ON tweets (id);")

In [16]:
cur.execute('SELECT * from tweets')
rows = cur.fetchall()
print(rows[0])

('377652254096228352', '224874450', '0101000020E6100000E1E5F1CD19B9424098331D328C785EC0', '2013-09-10 21:38:08-07', '@Tanner_Cortez hey checkout the website: http://t.co/LuktJ6hiws')


## 4.6 Number of tweets inside Alameda county

In [26]:
"""
TODO
Hint: use ST_Within() function
"""

query = "SELECT count(*) FROM tweets as tweets, ca_census_tract as ca WHERE ST_WITHIN(tweets.loc,ca.geom) AND ca.geoid10='06001';"
cur.execute(query)
count = cur.fetchall()
print('Number of tweets in Alameda:', count[0][0])

Number of tweets in Alameda: 16729


## 4.7 Number of tweets fall 10 km outside of Butte County?

In [3]:
"""
TODO
Hint: use ST_DWithin() function
"""
query = "SELECT count(*) FROM tweets as tweets, ca_census_tract as ca\
        WHERE ca.geoid10='06007' \
        AND NOT ST_Dwithin(tweets.loc,ca.geom, 10000, True);"

cur.execute(query)
count = cur.fetchall()

print('Number of tweets outside 10km of Butte:', count[0][0]))

Number of tweets outside 10km of Butte: 99094
